In [1]:
import numpy as np
import pandas as pd
import os
import os.path as opath
import ast
import pickle


from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<div class="span5 alert alert-info">
<h3>Note book details</h3>

<p> This notebook is part 1 of 2 for data wrangling for <b>Mortgage client segmentation</b> project.</p>

<p> Notes.</p>

<ol>
<li>  The file read the multiple csv files as input for each month </li> 
<li>  The file needs to be run manually for each month as files for following reason 
    <ul>
     <li> The files are available at end of months</li> 
    <li>  Processing each month is time consuming and resource extensive process on local computer </li> 
    </ul>
</li> 
<li>  The files are pickeld and csv to use further in project . This make easier to use data in later stages</li> 
    
</ol>
</div>

In [2]:
pd.set_option('display.max_colwidth', -1)
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
homeloancol= ['Year',
'RespondentID',
'AgencyCode',
'LoanType',
'PropertyType',
'LoanPurpose',
'Occupancy',
'LoanAmount',
'Preapproval',
'ActionType',
'MSA',
'StateCode',
'CountyCode',
'CensusTractNumber',
'ApplicantEthnicity',
'CoApplicantEthnicity',
'ApplicantRace',
'ApplicantRace1',
'ApplicantRace2',
'ApplicantRace3',
'ApplicantRace4',
'CoApplicantRace',
'CoApplicantRace1',
'CoApplicantRace2',
'CoApplicantRace3',
'CoApplicantRace4',
'ApplicantSex',
'CoApplicantSex',
'ApplicantIncome',
'PurchaserType',
'DenialReason1',
'DenialReason2',
'DenialReason3',
'RateSpread',
'HOEPAStatus',
'LienStatus',
'EditStatus',
'SequenceNumber',
'Population',
'MinorityPopulationPercen',
'FFIECMedianFamily',
'TracttoMSA',
'NumberofOwnerOccupied',
'Numberof1_To',
'ApplicationDateIndicator'
]

picklefilename='df_alldata_hmda.sa'

In [5]:
# based on inital data wrangling step the data will be removed for following columns
def removeDatatForNullColumns(df):
    #removed all rows without loan amount
    #df_alldata1=df_alldata[df_alldata['LoanAmount'].isnull()==False]
    df=df.drop(df[df['LoanAmount'].isnull()].index)

    # remove all where no geographic info available
    df=df.drop(df[df['MSA'].isnull() & df['StateCode'].isnull() & df['CountyCode'].isnull()].index)

    # remove all with no ApplicantIncome
    df=df.drop(df[df['ApplicantIncome'].isnull()].index)

    # remove all status 3,4,5 
    #df=df.drop(df[df.ActionType==3].index)
    df=df.drop(df[df.ActionType==4].index)
    df=df.drop(df[df.ActionType==5].index)
    return df

In [6]:
# reading all files in monthly folder
df_alldata =pd.DataFrame()
folderpath='data/hmda/'

for root,_,files in os.walk(folderpath):
    for f in files:
        fullpath=opath.join(root,f)
        df_file=pd.read_csv(fullpath, index_col=False,sep=',',names=homeloancol)
        df_alldata= df_alldata.append(df_file,ignore_index=True)
        df_alldata=removeDatatForNullColumns(df_alldata)
        print('file complete : '+f + ' no of rows : '+ str(len(df_file)) + ' no of columns :'+str(len(df_file.columns)))
#df_alldata.set_index('id').sort_index()
#df_alldata.columns=homeloancol

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,10,12,13,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


file complete : 2015HMDALAR - National.csv no of rows : 14374184 no of columns :45
file complete : 2016HMDALAR - National.csv no of rows : 16332987 no of columns :45
file complete : 2017_public_lar.csv no of rows : 14285496 no of columns :45


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: invalid continuation byte

### Check colums

In [7]:
df_alldata.head()

,Year,RespondentID,AgencyCode,LoanType,PropertyType,LoanPurpose,Occupancy,LoanAmount,Preapproval,ActionType,...,LienStatus,EditStatus,SequenceNumber,Population,MinorityPopulationPercen,FFIECMedianFamily,TracttoMSA,NumberofOwnerOccupied,Numberof1_To,ApplicationDateIndicator
0,2015,0000001281,3,1,1,3.0,1.0,222.0,3.0,3.0,...,1.0,6,343.0,4831,17.93,89600,117.52,1374,1797,0.0
1,2015,0000001281,3,1,1,3.0,2.0,188.0,3.0,3.0,...,1.0,6,345.0,5364,16.63,89600,111.23,1593,1754,0.0
2,2015,0000001281,3,1,1,3.0,2.0,216.0,3.0,1.0,...,1.0,6,509.0,1988,52.31,89600,58.72,149,327,0.0
3,2015,0000001281,3,1,1,3.0,1.0,219.0,3.0,1.0,...,1.0,6,630.0,3458,17.35,89600,112.09,998,1268,0.0
4,2015,0000001281,3,1,1,3.0,2.0,188.0,3.0,1.0,...,1.0,6,635.0,5364,16.63,89600,111.23,1593,1754,0.0


In [8]:
df_alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36384615 entries, 0 to 36384614
Data columns (total 45 columns):
Year                        object
RespondentID                object
AgencyCode                  object
LoanType                    object
PropertyType                object
LoanPurpose                 float64
Occupancy                   float64
LoanAmount                  float64
Preapproval                 float64
ActionType                  float64
MSA                         object
StateCode                   float64
CountyCode                  object
CensusTractNumber           object
ApplicantEthnicity          float64
CoApplicantEthnicity        float64
ApplicantRace               float64
ApplicantRace1              object
ApplicantRace2              object
ApplicantRace3              object
ApplicantRace4              object
CoApplicantRace             float64
CoApplicantRace1            object
CoApplicantRace2            object
CoApplicantRace3            object


In [9]:
# set up some field as category
for col in ['PropertyType', 'LoanPurpose', 'Occupancy', 'ApplicantEthnicity','CoApplicantEthnicity',
            'ApplicantRace','CoApplicantRace','ApplicantSex','CoApplicantSex','PurchaserType']:
    df_alldata[col] = df_alldata[col].astype('category')

In [10]:
df_alldata=df_alldata.replace('NA   ',np.NaN)
df_alldata=df_alldata.replace('NA  ',np.NaN)
df_alldata=df_alldata.replace('NA ',np.NaN)
df_alldata=df_alldata.replace('NA',np.NaN)

In [11]:
for col in ['MSA','CountyCode','ApplicantIncome']:
    #df_alldata[col]=df_alldata[col].fillna(0, inplace=True)
    df_alldata[col]=pd.to_numeric(df_alldata[col], downcast='integer')

In [12]:
df_alldata=removeDatatForNullColumns(df_alldata)

### Pickle and save the single monthly file for further processing 

In [13]:
# get the pickel file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
#picklefilename='df_alldata_hmda'
print(picklefilename)
#f1.info()

df_alldata_hmda.sa


In [14]:
# create pickle file for further use 
pickle.dump(df_alldata,open(picklefilename,'wb'), protocol=4)

In [15]:
# check the some other fields if they have siginificant data
#df_alldata[df_alldata[colname].isnotnull()]#
for col in homeloancol:#colnames:
    df_alldata.groupby(col).size()  

Year
2015    9999844 
2016    11513538
2017    10116644
dtype: int64

RespondentID
854           5   
861           1   
893           29  
1049          3   
1060          1   
1074          2   
1148          19  
1199          1   
1213          54  
1253          8   
1309          4   
1493          5   
1520          1   
1644          1   
1682          208 
1696          399 
1701          9   
1711          166 
1718          1018
1733          240 
1785          45  
1815          593 
1821          13  
1830          42  
1833          1   
1837          3   
1843          27  
1845          302 
1866          2   
1870          18  
              ..  
99-0035093    443 
99-0035281    128 
99-0050028    4302
99-0057039    4   
99-0057378    26  
99-0057942    9   
99-0058041    10  
99-0058081    7   
99-0058171    72  
99-0058179    17  
99-0058236    54  
99-0058313    7   
99-0058815    1   
99-0059010    428 
99-0090179    210 
99-0090283    94  
99-0355821    23  
99-0564605    1795
99-0617024    109 
99-0700016    35  
99-0700962    70  

AgencyCode
1    1423774 
2    958466  
3    2810069 
5    2452731 
7    14228992
9    9755994 
dtype: int64

LoanType
1    23953286
2    4853706 
3    2325292 
4    497742  
dtype: int64

PropertyType
1    30662265
2    967761  
3    0       
dtype: int64

...

FFIECMedianFamily
15800.0     1182 
18000.0     740  
18400.0     1265 
18800.0     2270 
20500.0     2810 
20700.0     1557 
23600.0     633  
25900.0     24413
27000       11217
37900.0     5835 
38200       6027 
40300.0     11086
41800       12477
42200.0     2814 
42800.0     4020 
43200       2663 
43300       2579 
43400       2290 
43900       1535 
44000       10363
44500       11210
44600       8502 
44700       12702
45000.0     7886 
45200       16705
45300.0     20046
45600.0     32966
45700.0     26442
45800       3930 
45900.0     19928
            ...  
00085600    2386 
00085700    64152
00085900    585  
00086700    2628 
00086800    6743 
00086900    1139 
00087000    5162 
00087100    25   
00087300    6154 
00087800    5555 
00087900    9564 
00089200    670  
00089300    5813 
00089600    21969
00089700    7251 
00090000    819  
00090300    2015 
00091000    5913 
00091600    4354 
00093000    4035 
00093800    2933 
00094800    4119 
00097000    3672 
00099400  

TracttoMSA
0.0       9884 
3.71      3    
6.32      3    
6.35      64   
6.63      3    
6.87      35   
7.72      5    
7.84      1    
8.23      3    
8.68      8    
8.88      8    
9.56      1    
9.6       5    
9.75      15   
9.94      2    
10.02     1    
10.04     6    
10.26     2    
10.31     64   
10.44     2    
10.48     7    
10.86     4    
10.9      23   
10.91     5    
11.18     2    
11.36     10   
11.42     3    
11.56     7    
11.65     8    
11.7      13   
          ..   
364.11    128  
364.66    64   
365.12    376  
366.38    168  
367.61    3700 
371.06    477  
372.10    137  
372.16    1106 
374.27    212  
375.83    162  
376.22    197  
376.43    265  
377.16    161  
377.55    255  
381.32    280  
384.65    4    
386.22    160  
387.84    132  
387.92    130  
388.15    52   
391.25    590  
394.69    365  
397.45    103  
399.23    15   
403.39    502  
405.70    1462 
425.35    6    
475.44    194  
499.35    42   
NA        84198
Length: 36509

NumberofOwnerOccupied
0.0         1056 
2.0         5    
3           18   
4           265  
5           64   
6           95   
7           98   
8           243  
9           164  
10          180  
11          74   
12          104  
13          66   
14          88   
15          102  
16          174  
17          121  
18          274  
19          65   
20          85   
21          35   
22          130  
23          148  
24          226  
25          160  
26          142  
27          53   
28          613  
29          343  
30          85   
            ..   
00004781    4234 
00004792    17   
00004820    851  
00004867    1292 
00004911    2272 
00004920    2883 
00004977    344  
00005087    1083 
00005092    567  
00005137    1246 
00005149    2295 
00005176    1983 
00005474    2072 
00005590    4008 
00005627    1962 
00005792    2707 
00005826    2168 
00005877    3994 
00005978    7284 
00006018    4097 
00006083    1553 
00006133    42   
00006293    950  
000064

Numberof1_To
0.0         753  
3           4    
4.0         19   
5.0         72   
6           253  
7           58   
8           65   
9           265  
10          295  
11          754  
12          354  
13          452  
14          152  
15          66   
16          302  
17          512  
18          163  
19          446  
20          237  
21          421  
22          195  
23          970  
24          318  
25          449  
26          461  
27          1195 
28          283  
29          400  
30          216  
31          400  
           ...   
00006103    2272 
00006142    1060 
00006239    1    
00006254    1246 
00006263    125  
00006272    3223 
00006298    88   
00006345    356  
00006362    683  
00006393    960  
00006428    495  
00006517    1091 
00006522    1118 
00006576    380  
00006587    259  
00006605    111  
00006667    4097 
00006862    4931 
00006872    2072 
00006908    680  
00007066    2391 
00007116    1529 
00007204    1018 
00007504    617

ApplicationDateIndicator
0.0    19700000
1.0    23      
2.0    1813359 
dtype: int64

In [16]:
#Extract minimum columns
homeloancolreq= ['Year',
'PropertyType',
'LoanPurpose',
'Occupancy',
'LoanAmount',
'ActionType',
'MSA',
'StateCode',
'CountyCode',
'ApplicantEthnicity',
'CoApplicantEthnicity',
'ApplicantRace',
'CoApplicantRace',
'ApplicantSex',
'CoApplicantSex',
'ApplicantIncome',
'PurchaserType',
]
df_selecteddata=df_alldata.loc[:,homeloancolreq]

In [17]:
#df_selecteddata.info()
picklefilename='df_selectdata_hmda.sa'

In [18]:
df_selecteddata.head()

,Year,PropertyType,LoanPurpose,Occupancy,LoanAmount,ActionType,MSA,StateCode,CountyCode,ApplicantEthnicity,CoApplicantEthnicity,ApplicantRace,CoApplicantRace,ApplicantSex,CoApplicantSex,ApplicantIncome,PurchaserType
0,2015,1,3.0,1.0,222.0,3.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,115.0,0.0
1,2015,1,3.0,2.0,188.0,3.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,115.0,0.0
2,2015,1,3.0,2.0,216.0,1.0,11260.0,2.0,20.0,2.0,5.0,5.0,8.0,1.0,5.0,230.0,7.0
3,2015,1,3.0,1.0,219.0,1.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,145.0,1.0
4,2015,1,3.0,2.0,188.0,1.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,145.0,1.0


### Pickle and save the single monthly file for further processing 

In [19]:
# get the pickel file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
#f1.info()

df_selectdata_hmda.sa


In [20]:
# create pickle file for further use 
pickle.dump(df_selecteddata,open(picklefilename,'wb'), protocol=4)

In [20]:
# check pickle file
upickle_df = pickle.load(open(picklefilename,"rb"))

In [21]:
upickle_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25179402 entries, 0 to 35398216
Data columns (total 17 columns):
Year                    int64
PropertyType            category
LoanPurpose             category
Occupancy               category
LoanAmount              float64
ActionType              int64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      category
CoApplicantEthnicity    category
ApplicantRace           category
CoApplicantRace         category
ApplicantSex            category
CoApplicantSex          category
ApplicantIncome         float64
PurchaserType           category
dtypes: category(10), float64(5), int64(2)
memory usage: 1.7 GB


In [22]:
df_newdata=upickle_df

In [23]:
df_newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25179402 entries, 0 to 35398216
Data columns (total 17 columns):
Year                    int64
PropertyType            category
LoanPurpose             category
Occupancy               category
LoanAmount              float64
ActionType              int64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      category
CoApplicantEthnicity    category
ApplicantRace           category
CoApplicantRace         category
ApplicantSex            category
CoApplicantSex          category
ApplicantIncome         float64
PurchaserType           category
dtypes: category(10), float64(5), int64(2)
memory usage: 1.7 GB


In [24]:
def checkforNullCol(colname,df):
    return df[df[colname].isnull()]#[colname,homeloancol]

In [25]:
def checkforNanCol(colname,df):
    return df[np.isnan(df[colname])]

In [29]:
t=df_newdata.columns

In [30]:
for col in t:
    
    df=checkforNullCol(col,df_newdata)
    if (len(df)>0):
        print(col + ' Null len :'+str(len(df)) )
    #df1=checkforNanCol(col)
    #if (len(df1)>0):
        #print(col + ' Nan len :'+str(len(df1)) )
        

MSA Null len :2552710
CountyCode Null len :17446


In [31]:
df_newdata.groupby('ActionType').size() 

ActionType
1    21287511
2    1125929 
6    2762256 
7    1952    
8    1754    
dtype: int64

In [32]:
print(len(df_selecteddata))
print(len(df_alldata))

25179402
25179402
